# Goal: simulate market dataset such that we can correctly estimate coefficients later on 



In [17]:
import import_ipynb
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

n_market = 50
n_product = 5
n_consumer = 100
market_variance = 0.5 #amount of variation between markets 
endogenous_noise = 0.5 #how noise to generate price 

#structural parameters 
true_alpha = 3 
true_beta = 7 
true_sigma_alpha = 0.7 
true_sigma_beta = 0.3
market_variance = 0.3 #amount of variation between markets 

In [19]:
all_markets = []
for market in range(n_market):
    x = np.random.uniform(0, 1, n_product)
    instrument = np.random.uniform(0, 10, n_product)

    # generate market shares and price variables endogenously 
    price = f.price_gen(x, instrument, endogenous_noise)
    inner_utility = f.utility_gen([true_alpha, true_beta], [true_sigma_alpha, true_sigma_beta], x, price)
    inner_market_utility = f.market_gen(inner_utility, market_variance)
    inner_prob_choice = f.choice_prob(pd.DataFrame(inner_market_utility))
    # generate market share directly from the choice probabilities
    outer_market_share = inner_prob_choice.sum(axis=0)/n_consumer

    market_j = pd.DataFrame({
        'market_id': market,
        'product_id': list(range(n_product)),
        'x': x, 
        'instrument': instrument, 
        'price': price,
        'share': outer_market_share, 
        'share_not_buy': 1 - sum(outer_market_share)
    })
    # print(market)
    all_markets.append(market_j)

print(all_markets[:50])
export_dataset = pd.concat(all_markets, ignore_index=True)
export_dataset.to_csv('simulated_market_data.csv', index=False)

[   market_id  product_id         x  instrument     price     share  \
0          0           0  0.804249    7.694159  2.447689  0.011046   
1          0           1  0.975991    9.772533  3.084254  0.010571   
2          0           2  0.991298    3.941569  1.567254  0.323098   
3          0           3  0.563043    4.159879  1.392711  0.027325   
4          0           4  0.327275    0.324548  0.261923  0.132424   

   share_not_buy  
0       0.495536  
1       0.495536  
2       0.495536  
3       0.495536  
4       0.495536  ,    market_id  product_id         x  instrument     price     share  \
0          1           0  0.455589    6.404394  2.575878  0.003768   
1          1           1  0.525766    5.291268  2.229740  0.012895   
2          1           2  0.939369    4.556634  2.251055  0.221322   
3          1           3  0.155135    1.378175  0.593134  0.076504   
4          1           4  0.480904    5.527920  2.282969  0.008746   

   share_not_buy  
0       0.676764  
1   